In [41]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline 

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [54]:
weather_df=pd.read_csv('/kaggle/input/weatherww2/Summary of Weather.csv')

In [55]:
#See some examples of what is the data
weather_df.head(5)

In [56]:
weather_df.info()

In [57]:
#Drop columns after column 9 as there is no description for their values 
drop_cols=weather_df.columns[9:]
weather_df.drop(drop_cols,axis=1,inplace=True)
weather_df.head()

In [58]:
#Set proper datatypes
weather_df['Precip']=pd.to_numeric(weather_df['Precip'],errors='coerce').fillna(0) #Non numeric characters represent T which stands for traces which is equivalent to 0
weather_df['Snowfall']=pd.to_numeric(weather_df['Snowfall'],errors='coerce').fillna(0)


In [59]:
#Get column descriptions
weather_df.info()

In [60]:
weather_df.isnull().sum()

In [61]:
#Check null values and the total percentage
missing_values=weather_df.isnull().sum()
missing_percentage=(missing_values*100)/weather_df.shape[0]
missing_df=pd.concat([missing_values,missing_percentage],axis=1,keys=['Missing_values','Percentage'])
missing_df.head(10)

In [62]:
#Remove the columns where more than 50% of data is missing
weather_df.drop(missing_df.query('Percentage > 50').index,axis=1,inplace=True)
#Drop columns STA and Date too as they are not required
weather_df.drop(['STA','Date'],axis=1,inplace=True)

In [63]:
print(weather_df['Snowfall'].value_counts())
#Drop the column Snowfall 
weather_df.drop(['Snowfall'],axis=1,inplace=True)


In [64]:
print(weather_df['Precip'].value_counts())
#Drop the column Precip as most of the values are 0 
weather_df.drop(['Precip'],axis=1,inplace=True)


In [65]:
#Once again check the description of dataframe
weather_df.describe(include='all',percentiles=[0.1,0.25,0.5,0.75,0.99])

In [ ]:
#Visualize the pair plot of the data
sns.pairplot(weather_df)

In [66]:
sns.heatmap(weather_df.corr())

In [67]:
#Now as we have cleaned the dataset we will find the regression maximum temprature from minimum temprature
#First we will carefully visualize data for both
min_temp=weather_df['MinTemp']
max_temp=weather_df['MaxTemp']

import matplotlib.pyplot as plt
%matplotlib inline
plt.scatter(min_temp,max_temp)

In [68]:
plt.hist(min_temp)

In [69]:
plt.hist(max_temp)

In [70]:
max_temp.plot(kind='box')

In [74]:
plt.boxplot(min_temp)

In [75]:
new_df[new_df['Min_temp'].duplicated(keep=False)]

In [76]:
min_temp.shape,max_temp.shape

In [77]:
from sklearn.model_selection import train_test_split

train_x,test_x,train_y,test_y=train_test_split(np.reshape(min_temp.values,(-1,1)),np.reshape(max_temp.values,(-1,1)),test_size=0.33)

In [78]:
from sklearn.linear_model import LinearRegression
model=LinearRegression().fit(train_x,train_y)

In [79]:
model.coef_,model.intercept_

In [80]:
#Check the fit on train dataset
plt.scatter(train_x,train_y)
plt.plot(train_x,model.predict(train_x),c='red')
plt.show()

In [81]:
#Check the prediction on test dataset
plt.scatter(test_x,test_y)
plt.plot(test_x,model.predict(test_x),c='red')
plt.show()